In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('boradcast').getOrCreate()

In [3]:
countries = {'IND':"India",'AUS':"Australia", 'JPN':"Japan"}

In [4]:
broadcast_countries = spark.sparkContext.broadcast(countries)

In [5]:
data = [(100,'Debit',1000.0,'IND'),(101,"Credit",2000.0,'IND'),
        (102,'Debit',3000.0,'AUS'),(103,'Credit',4000.0,'JPN'),
        (104,'Debit',5000.0,'IND'),(105,'Credit',6000.0,'AUS')]

In [6]:
rdd = spark.sparkContext.parallelize(data)

In [7]:
rdd.collect()

[(100, 'Debit', 1000.0, 'IND'),
 (101, 'Credit', 2000.0, 'IND'),
 (102, 'Debit', 3000.0, 'AUS'),
 (103, 'Credit', 4000.0, 'JPN'),
 (104, 'Debit', 5000.0, 'IND'),
 (105, 'Credit', 6000.0, 'AUS')]

In [8]:
def country(code):
    return broadcast_countries.value[code]

In [9]:
rdd1 = rdd.map(lambda x : (x[0], x[1], x[2], country(x[3])))

In [10]:
rdd1.collect()

[(100, 'Debit', 1000.0, 'India'),
 (101, 'Credit', 2000.0, 'India'),
 (102, 'Debit', 3000.0, 'Australia'),
 (103, 'Credit', 4000.0, 'Japan'),
 (104, 'Debit', 5000.0, 'India'),
 (105, 'Credit', 6000.0, 'Australia')]

In [11]:
rdd2 = rdd1.groupBy(lambda x : (x[3], x[1])).mapValues(list)
rdd2.collect()

[(('Australia', 'Debit'), [(102, 'Debit', 3000.0, 'Australia')]),
 (('Australia', 'Credit'), [(105, 'Credit', 6000.0, 'Australia')]),
 (('India', 'Debit'),
  [(100, 'Debit', 1000.0, 'India'), (104, 'Debit', 5000.0, 'India')]),
 (('Japan', 'Credit'), [(103, 'Credit', 4000.0, 'Japan')]),
 (('India', 'Credit'), [(101, 'Credit', 2000.0, 'India')])]

In [12]:
rdd2.map(lambda x :(x[0],len(x[1]))).collect()

[(('Australia', 'Debit'), 1),
 (('Australia', 'Credit'), 1),
 (('India', 'Debit'), 2),
 (('Japan', 'Credit'), 1),
 (('India', 'Credit'), 1)]